In [5]:
#  APIs from Keras we will be using
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K

In [6]:
"""
Here we declare all important parameters we will be using
throughout the development such as dimensions of images,
path to train & validation datasets etc...
"""
img_width, img_height = 150, 150
train_data_dir = 'data/train'
validation_data_dir = 'data/validation'
nb_train_samples = 2000
nb_validation_samples = 800
epochs = 50
batch_size = 16

In [7]:
#  We need to check here how Keras.json is setup
#  Channels == 3 since we have RGB values in our pixels
#  channels_first means we img[0] indicates RGB images, img[1] is width of the image, img[2] is the height of the img
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

In [8]:
#  Instantiation of Keras Object for Convulation Neural Networks
model = Sequential()

#  the first layer needs to know what kind of shape to expect
#  next layers will automatically infer the dimensions
model.add(Conv2D(32, (3, 3), input_shape=input_shape))

#  here we specify function which filter will apply to the image 
model.add(Activation('relu'))

#  subsampling, we're specifying just the size of filter
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

Instructions for updating:
Colocations handled automatically by placer.


In [9]:
#  this is so-called dense layer. It takes data produced by ConvNet
#  and applies yet another set of layers to come up with final one value.. 0 or 1
model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [10]:
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])